# SELECT ... FROM

In [1]:
QUERY=(
'SELECT Name FROM `bigquery-public-data.pet_records.pets` '
'WHERE Animal = "Cat"'
'LIMIT 100')

In [2]:
#Using global_air_quality

from google.cloud import bigquery
client = bigquery.Client.from_service_account_json("./My Project 61217-9183df10355d.json")

In [4]:
dataset_ref = client.dataset( 'openaq', project = 'bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

In [10]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

global_air_quality


In [16]:
table_ref = dataset_ref.table('global_air_quality')
table = client.get_table(table_ref)
table.schema

[SchemaField('location', 'STRING', 'NULLABLE', 'Location where data was measured', ()),
 SchemaField('city', 'STRING', 'NULLABLE', 'City containing location', ()),
 SchemaField('country', 'STRING', 'NULLABLE', 'Country containing measurement in 2 letter ISO code', ()),
 SchemaField('pollutant', 'STRING', 'NULLABLE', 'Name of the Pollutant being measured. Allowed values: PM25, PM10, SO2, NO2, O3, CO, BC', ()),
 SchemaField('value', 'FLOAT', 'NULLABLE', 'Latest measured value for the pollutant', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'The datetime at which the pollutant was measured, in ISO 8601 format', ()),
 SchemaField('unit', 'STRING', 'NULLABLE', 'The unit the value was measured in coded by UCUM Code', ()),
 SchemaField('source_name', 'STRING', 'NULLABLE', 'Name of the source of the data', ()),
 SchemaField('latitude', 'FLOAT', 'NULLABLE', 'Latitude in decimal degrees. Precision >3 decimal points.', ()),
 SchemaField('longitude', 'FLOAT', 'NULLABLE', 'Longitude in d

In [17]:
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [20]:
QUERY= """
        SELECT city 
        FROM `bigquery-public-data.openaq.global_air_quality` 
        WHERE country = "US"
        """


In [21]:
# Submit the query to the dataset
client = bigquery.Client.from_service_account_json("./My Project 61217-9183df10355d.json")

In [22]:
query_job = client.query(QUERY)

In [25]:
# return the pandas dataframe
us_cities_df = query_job.to_dataframe()
us_cities_df.head()

,city
0,Houston
1,Houston
2,Houston
3,Houston
4,Houston


In [27]:
us_cities_df.city.value_counts().head()

Phoenix-Mesa-Scottsdale                     87
Houston                                     82
Los Angeles-Long Beach-Santa Ana            63
New York-Northern New Jersey-Long Island    60
San Francisco-Oakland-Fremont               58
Name: city, dtype: int64

In [30]:
# Selecting multiple columns
QUERY= """
        SELECT city, country 
        FROM `bigquery-public-data.openaq.global_air_quality` 
        WHERE country = "US"
        """

In [34]:
query_job = client.query(QUERY)
query_job.to_dataframe().head()

,city,country
0,Houston,US
1,Houston,US
2,Houston,US
3,Houston,US
4,Houston,US


In [35]:
# Select all the columns with *
QUERY= """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality` 
        WHERE country = "US"
        """

In [36]:
query_job = client.query(QUERY)
query_job.to_dataframe().head()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,Freeport South Avenue I C1012,Houston,US,so2,0.0001,2016-03-06 18:00:00+00:00,ppm,Texas,28.964430,-95.354830,1.0
1,Lake Jackson C1016,Houston,US,no2,-0.0004,2016-03-06 18:00:00+00:00,ppm,Texas,29.043758,-95.472946,1.0
2,Danciger C618,Houston,US,no2,0.0005,2016-03-06 17:00:00+00:00,ppm,Texas,29.144290,-95.756600,1.0
3,Danciger C618,Houston,US,o3,0.0460,2016-03-06 17:00:00+00:00,ppm,Texas,29.144290,-95.756600,1.0
4,Galveston 99th St. C1034/A320/X183,Houston,US,no2,0.0017,2016-03-06 19:00:00+00:00,ppm,Texas,29.254473,-94.861290,1.0


# Big datasets

- how much data a query can scan with QUeryJobConfig

In [41]:
dataset_ref = client.dataset('hacker_news', project = 'bigquery-public-data')
# API request -fetch dataset
dataset = client.get_dataset(dataset_ref)

In [45]:
query = """
        SELECT score , title
        FROM `bigquery-public-data.hacker_news.full`
        WHERE type = 'job'

        """

In [46]:
# Create a QueryJobConfig object to estimate size of query without runningit

In [49]:
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query_job = client.query(query, job_config=dry_run_config)
print('this query will process {} bytes'. format(dry_run_query_job.total_bytes_processed))

this query will process 367900255 bytes


In [50]:
ONE_MB = 1000*1000

In [51]:
safe_config= bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

In [54]:
safe_config_job = client.query(query, job_config=safe_config)